In [32]:
from param_def import chi,sigma,prop_gp,capacities_rate,prop_all_g_prefer,capacities,lambdas
import numpy as np
from util import welfare_metrics,student_by_col,bayes_update_grade,grades_gr_to_grades_col,utility_by_col
from itertools import compress
# Import HyperOpt Library
from hyperopt import tpe, hp, fmin
import numpy as np
from BayesEst import bayesian_est,anal_cond_exp
from scipy.optimize import fsolve
from scipy.stats import norm, multivariate_normal
from param_def import chi,sigma,lambdas

In [33]:
print(capacities_rate)

[0.2, 0.2]


In [34]:
grade_estimated_gr = []
for i in range(len(chi)):
    grade_estimated_gr.append(np.load(f'grade_estimated_gr{i+1}.npy'))
grade_estimated = grades_gr_to_grades_col(grade_estimated_gr)
#%%
stud_pref = []
for i in range(len(chi)):
    stud_pref.append(np.load(f'stud_pref_gr{i+1}.npy').tolist())

In [35]:
def cdf(x, sigma):
    return norm.cdf(x, scale = sigma)

In [36]:
def cdfm(x, y, sigma, rho):
    return multivariate_normal([0, 0], [[sigma**2, sigma**2 * rho], [sigma**2 * rho, sigma**2]]).cdf(np.array([x,y]))

In [37]:
def cdfmsi(x, y, sigma, rho):
    return norm.cdf(y, scale = sigma) - multivariate_normal([0, 0], [[sigma**2, sigma**2 * rho], [sigma**2 * rho, sigma**2]]).cdf(np.array([x,y]))

In [38]:
def cdfmis(x, y, sigma, rho):
    return norm.cdf(x, scale = sigma) - multivariate_normal([0, 0], [[sigma**2, sigma**2 * rho], [sigma**2 * rho, sigma**2]]).cdf(np.array([x,y]))

In [39]:
def market_clear(Pa, Pb, prop, capA, capB, prefi, prefii, sigmai, sigmaii, cori, corii):
    f1 = prop*prefi*(1 - cdf(Pa, sigmai)) + (1 - prop)*prefii*(1 - cdf(Pa, sigmaii)) + prop*(1 - prefi)*cdfmsi(Pa, Pb, sigmai, cori) + (1 -prop)*(1 - prefii)*cdfmsi(Pa, Pb, sigmaii, corii) - capA
    f2 = prop*(1 - prefi)*(1 - cdf(Pb, sigmai)) + (1 - prop)*(1 - prefii)*(1 - cdf(Pb, sigmaii)) + prop*prefi*cdfmis(Pa, Pb, sigmai, cori) + (1 -prop)*prefii*cdfmis(Pa, Pb, sigmaii, corii) - capB
    return f1, f2

In [40]:
def objective_bads(params):
    Pa,Pb = params
    f1,f2 = market_clear(Pa,Pb,prop_gp[0],capacities_rate[0],capacities_rate[1],
                  prop_all_g_prefer[0],prop_all_g_prefer[1],sigma_i,sigma_ii,cor_i,cor_ii)
    # return np.abs(f1) + np.abs(f2)
    return f1**2+f2**2

In [41]:
# def solv_mc(prop, capA, capB, prefi, prefii, sigmai, sigmaii, cori, corii):
#     func = lambda P: market_clear(P[0], P[1], prop, capA, capB, prefi, prefii, sigmai, sigmaii, cori, corii)
#     return fsolve(func, x0 = np.array([0, 0]))

# P_A,P_B = solv_mc(prop_gp[0],capacities_rate[0],capacities_rate[1],
#                   prop_all_g_prefer[0],prop_all_g_prefer[1],sigma_i,sigma_ii,cor_i,cor_ii)


In [42]:
sigma_i = np.sqrt(chi[0]**2 + sigma[0]**2)
sigma_ii = np.sqrt(chi[1]**2 + sigma[1]**2)
cor_i = (chi[0]**2 + lambdas[0]**2)/sigma_i**2
cor_ii = (chi[1]**2 + lambdas[1]**2)/sigma_ii**2

In [43]:
from pybads import BADS

lower_bounds = np.array([-10, -10])
upper_bounds = np.array([10, 10])
plausible_lower_bounds = np.array([-5, -5])
plausible_upper_bounds = np.array([5, 5])
x0 = np.array([0, 0]);        # Starting point
# %%
bads = BADS(objective_bads, x0, lower_bounds, upper_bounds, plausible_lower_bounds, plausible_upper_bounds)
optimize_result = bads.optimize()

# %%
x_min = optimize_result['x']
fval = optimize_result['fval']
print(f"BADS minimum at: x_min = {x_min.flatten()}, fval = {fval:.4g}")
print(f"total f-count: {optimize_result['func_count']}, time: {round(optimize_result['total_time'], 2)} s")

Beginning optimization of a DETERMINISTIC objective function

 Iteration    f-count         f(x)           MeshScale          Method             Actions
     0           2       0.0336703               1                                 Uncertainty test
     0           6       0.0336703               1         Initial mesh            Initial points


     0          10       0.0336703             0.5         Refine grid             Train
     1          12       0.0261156             0.5     Incremental search (ES-wcm)        
     1          18     0.000460321            0.25         Refine grid             Train
     2          26     0.000460321           0.125         Refine grid             Train
     3          27      2.4725e-05           0.125     Incremental search (ES-wcm)        
     3          28     2.24125e-05           0.125     Incremental search (ES-wcm)        
     3          30     6.97087e-07           0.125     Incremental search (ES-wcm)        
     3          34     6.97087e-07          0.0625         Refine grid             
     4          35     2.96627e-08          0.0625     Incremental search (ES-wcm)        
     4          37     1.96759e-08          0.0625     Incremental search (ES-ell)        
     4          42     1.96759e-08        0.015625         Refine grid             Train
     5        

In [44]:
cutoff_values = list(x_min)

In [45]:
welfare_metrics(cutoff_values,grade_estimated,stud_pref)

Proportion of students in group 0 with no offer 0.59
Proportion of students in group 0 with only a second preference offer 0.13
Proportion of students in group 0 with first choice offer 0.28
Proportion of students in group 1 with no offer 0.60
Proportion of students in group 1 with only a second preference offer 0.12
Proportion of students in group 1 with first choice offer 0.28


In [46]:
student_by_col(cutoff_values,grade_estimated,stud_pref)

Number of student admitted to college 1 in group 0: 1650
Number of student admitted to college 2 in group 0: 1637
Number of student admitted to college 1 in group 1: 2389
Number of student admitted to college 2 in group 1: 2398


(4039, 4035)

In [47]:
utility_by_col(cutoff_values,grade_estimated,stud_pref)

Total A utility: 12782.275207483755
A utility for student with first choice: 7977.8794540677245
A utility for student with second choice: 4804.395753416031
Total B utility: 13864.972455693087
B utility for student with first choice: 11218.112296953987
B utility for student with second choice: 2646.8601587391004


True

In [48]:
print(capacities)

[4000, 4000]


In [49]:
import pandas as pd
df1 = pd.DataFrame({'A':grade_estimated[0][0],'B':grade_estimated[1][0],'pref':np.array(stud_pref[0]).T[0]})
df2 = pd.DataFrame({'A':grade_estimated[0][1],'B':grade_estimated[1][1],'pref':np.array(stud_pref[1]).T[0]})

Partially update

In [19]:
new_cutoff_values = [1.77327044,0.69884685]
new_cutoff_values

[1.77327044, 0.69884685]

In [20]:
updated_grade_estimated = bayes_update_grade(new_cutoff_values[0],new_cutoff_values[1],grade_estimated,stud_pref,chi,sigma,lambdas,bayes_type='right_partial')

In [21]:
##    
# grade_estimated should have the following form 
# [[[-3.62656622994722, 2.1811620120023885, 
# -11.563939588668866, 9.522766463940787], 
# [-2.140950614292475, -3.1545339753546977, 
# 0.2686527056082677, -0.6574626041532636, 
# -2.252868772168457, 1.8937660992816987]], 
# [[-5.268780813142571, -0.8191620654894043, 
# -9.60125782299109, 7.9126466724151605], 
# [-3.9756947771758844, -2.6409442009387023, 
# 4.086647894727917, -1.357128391535272, 
# 0.8661128454577303, 0.9586986061854232]]] 
# with 2 groups, 2 colleges, group 1 has 4 students, 
# group 2 has 6 students.
# updated_grade_estimate should have the same form as grade_estimated

In [22]:
welfare_metrics(new_cutoff_values,updated_grade_estimated,stud_pref)

Proportion of students in group 0 with no offer 0.59
Proportion of students in group 0 with only a second preference offer 0.18
Proportion of students in group 0 with first choice offer 0.23


Proportion of students in group 1 with no offer 0.61
Proportion of students in group 1 with only a second preference offer 0.18
Proportion of students in group 1 with first choice offer 0.22


In [23]:
student_by_col(new_cutoff_values,updated_grade_estimated,stud_pref)

Number of student admitted to college 1 in group 0: 1643
Number of student admitted to college 2 in group 0: 1648
Number of student admitted to college 1 in group 1: 2362
Number of student admitted to college 2 in group 1: 2353


(4005, 4001)

In [24]:
utility_by_col(cutoff_values,updated_grade_estimated,stud_pref)

Total A utility: 18233.87382707239
A utility for student with first choice: 7968.518719899028
A utility for student with second choice: 10265.355107173364
Total B utility: 3549.0399578564466
B utility for student with first choice: 897.3264536034676
B utility for student with second choice: 2651.713504252979


True

Update all

In [27]:
new_cutoff_values = [1.57124967,0.40021554]
new_cutoff_values

[1.57124967, 0.40021554]

In [28]:
updated_grade_estimated = bayes_update_grade(new_cutoff_values[0],new_cutoff_values[1],grade_estimated,stud_pref,chi,sigma,lambdas,bayes_type='right_all')

In [29]:
welfare_metrics(new_cutoff_values,updated_grade_estimated,stud_pref)

Proportion of students in group 0 with no offer 0.59
Proportion of students in group 0 with only a second preference offer 0.13
Proportion of students in group 0 with first choice offer 0.28
Proportion of students in group 1 with no offer 0.60
Proportion of students in group 1 with only a second preference offer 0.12
Proportion of students in group 1 with first choice offer 0.27


In [30]:
student_by_col(new_cutoff_values,updated_grade_estimated,stud_pref)

Number of student admitted to college 1 in group 0: 1643
Number of student admitted to college 2 in group 0: 1614
Number of student admitted to college 1 in group 1: 2392
Number of student admitted to college 2 in group 1: 2356


(4035, 3970)

In [31]:
utility_by_col(cutoff_values,updated_grade_estimated,stud_pref)

Total A utility: 18286.266596586414
A utility for student with first choice: 7968.518719899028
A utility for student with second choice: 10317.747876687386
Total B utility: 926.2828545092809
B utility for student with first choice: 844.8475523195892
B utility for student with second choice: 81.4353021896917


True